In [38]:
import pandas as pd
import numpy as np

In [39]:
# this should probably be on a seperate cell so I don't constantly reload the dataframes
# constants so I don't have to keep changing names
BS = 'business'
CH = 'checkin'
TI = 'tip'
RW = 'review'
US = 'user'

# load files in (right now I'm skipping the reviews part cause it's big and my work laptop doens't have the space)
fDefPath = 'reviews/yelpReviews/yelp_academic_dataset_'
fYelpNames = [BS, CH, TI]


In [40]:
fdf = {}

# load the datasets
for name in fYelpNames:
    fPath = f'{fDefPath}{name}.json'
    fdf[name] = pd.read_json(fPath, lines=True)

for name in fYelpNames:
    print(fdf[name].columns)

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')
Index(['business_id', 'date'], dtype='object')
Index(['user_id', 'business_id', 'text', 'date', 'compliment_count'], dtype='object')


In [72]:
bsdf = fdf[BS].dropna(subset=["categories"]).copy()
bsdf['categories'] = bsdf['categories'].str.lower()
bsdf.loc[:,"categoryList"] = bsdf.loc[:,"categories"].str.split(", ")

# Flatten and count unique categories
allCategories = bsdf["categoryList"].explode().str.lower()
categoryCounts = allCategories.value_counts()

# Preview common categories
print(categoryCounts.head(50))

categoryList
restaurants                  52268
food                         27781
shopping                     24395
home services                14356
beauty & spas                14292
nightlife                    12281
health & medical             11890
local services               11198
bars                         11065
automotive                   10773
event planning & services     9895
sandwiches                    8366
american (traditional)        8139
active life                   7687
pizza                         7093
coffee & tea                  6703
fast food                     6472
breakfast & brunch            6239
american (new)                6097
hotels & travel               5857
home & garden                 5799
fashion                       5739
burgers                       5636
arts & entertainment          5434
auto repair                   5433
hair salons                   5046
nail salons                   4621
mexican                       4600
italian

In [74]:
# finding what american new means
bsdf[bsdf["categories"].str.contains(r"american \(new\)", na=False)].name.head(50)


58                                      Roman Forum
61                                       Twin Peaks
64       Doc Magrogan's Oyster House - West Chester
79                          Top Shelf Sports Lounge
99                           Ricardo's Italian Cafe
110                                Sunset 44 Bistro
181     Gaylord Opryland Resort & Convention Center
198                                    Aussie Grill
231                     Eat Mah Taco @ Pal's Lounge
247                        The Libertine Liquor Bar
372                                        The Love
386                                Ryan Christopher
443                                The Twisted Tail
461                                     Local Links
476                                    Garage Pizza
502                            NY Bagel Cafe & Deli
524                                Whetstone Tavern
537                                      The Bricks
557                             Reno Downtown Joint
559         

In [110]:
# create a manual category for now
categoryMap = {
    'restaurants' : {'restaurants', 'food', 'sandwiches', 'american (new)', 'american (traditional)'},
    'retail' : {'shopping', 'home & garden', 'fashion', 'real estate', 'grocery'},
    'beauty' : {'beauty & spas', 'hair salons', 'nail salons'},
    'auto' : {'automotive', 'auto repair'},
    'services' : {'home services', 'local services', 'event planning & services'},
    'fitness' : {'active life', 'fitness & instruction'},
    'health' : {'health & medical', 'doctors', 'dentist'},
    'pet services' : {'veternarians', 'pet services', 'pets'},
    'nightlife' : {'nightlife', 'bars'},
    'travel' : {'hotels & travel', 'hotels', 'tours'},
    'religious' : {'religious organizations', 'synagogous'},
    'entertainment' : {'festivals', 'arts & entertainment', 'television', 'mass media'},
    'finance' : {'banks & credit unions', 'financial services'},
    'legal' : {'dui law', 'lawyers'},
    'government' : {'public services & government', 'libraries',},
    'education' : {'education'},
    'employment services' : {'professional services', 'employment agencies'}
}

In [111]:
def matchBroadCategory(catList):
    for broadCat, keywords in categoryMap.items():
        if any(cat in keywords for cat in catList):
            return broadCat
    return "other"

bsdf["broadCategory"] = bsdf["categoryList"].apply(matchBroadCategory)


In [113]:
bsdf[bsdf["broadCategory"].str.contains("other", na=False)].head(81)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,categoryList,broadCategory
2152,klGvxx2XmKkd-9mdRZoGVA,The Comcast Experience HD Video Wall,1701 Arch St,Philadelphia,PA,19103,39.954477,-75.168542,4.0,50,1,{'GoodForKids': 'True'},local flavor,None,[local flavor],other
3569,mnRgVArZmYhkz5zQJ6zhtw,China Lights,1 Palm Dr,New Orleans,LA,70124,29.993302,-90.098153,4.5,9,1,{'GoodForKids': 'True'},local flavor,None,[local flavor],other
3964,ST6l2UHi9lnr2Ktx5yTOTw,Elite Event - Board & Brush,"7530 Longley Ln, Ste 104",Reno,NV,89511,39.456281,-119.777466,5.0,34,0,None,"local flavor, yelp events",None,"[local flavor, yelp events]",other
4915,ig0VZJWIzYmKy8mO4IYqsQ,St. Louis Art Fair,Forsyth Blvd,Saint Louis,MO,63105,38.650184,-90.337572,3.5,15,1,{'GoodForKids': 'True'},local flavor,None,[local flavor],other
7088,lIooZerqwDWdARQOhng0dQ,Chromatic Gate,721 Ecabrillo Blvd,Santa Barbara,CA,93103,34.416183,-119.675267,4.5,21,1,{'GoodForKids': 'True'},"local flavor, public art","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...","[local flavor, public art]",other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143725,ezLDUMrSdXBCk6QOqOrPwA,Adult Swim,,Indianapolis,IN,,39.811099,-86.159835,4.0,9,1,{'GoodForKids': 'False'},local flavor,None,[local flavor],other
146674,fLYbnrj7Tnds-N7hZW3MDA,Fantasies In Chocolate,Grand Sierra Resort,Reno,NV,89501,39.524314,-119.780723,4.0,8,1,{'GoodForKids': 'False'},local flavor,None,[local flavor],other
147510,b8T4pPjcLzmQg5GAGTGZlg,Yelp Elite Week with Banh Mi Boys,"5001 Airline Dr, Ste B",Metairie,LA,70001,29.976384,-90.186617,5.0,39,1,None,"yelp events, local flavor",None,"[yelp events, local flavor]",other
148249,URErL0Vd3o6DXA9ME60yJQ,Yelp Presents: Breaking Bread and Badges at So...,11454 Jasper Ave,Edmonton,AB,T5K 0M1,53.541327,-113.518809,4.5,13,1,None,"yelp events, local flavor",None,"[yelp events, local flavor]",other
